## Notebook for proliferating tumor cells identification using `ikarus`

- **Developed by**: Anna Maguza
- **Institute of Computational Biology - Computational Health Centre - Helmholtz Munich**
- 31st May 2023

### Import packages

In [1]:
import urllib.request
import anndata
import pandas as pd
from pathlib import Path
from ikarus import classifier, utils, data
import scanpy as sc

### Data Upload

In [2]:
paths = [
    Path("/Users/anna.maguza/Desktop/Data/Gut_project/Joanito_cancer/anndata/")
]
names = [
    "Joanito"
]
adatas = {}
for path, name in zip(paths, names):
    adatas[name] = anndata.read_h5ad(path / "Joanito_raw_anndata_tumor_cells.h5ad")
    # Uncomment to perform preprocessing. Here, the loaded anndata objects are already preprocessed. 
    adatas[name] = adatas[name][adatas[name].obs['Cell Type'] == 'Epithelial',:]
    adatas[name] = data.preprocess_adata(adatas[name])
    adatas[name].var['gene_symbol'] = adatas[name].var.index

/Users/anna.maguza/miniconda3/envs/ikarus_env/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [3]:
### Upload cancer data
input_cancer = '/Users/anna.maguza/Desktop/Data/Gut_project/Joanito_cancer/anndata/Joanito_raw_anndata_tumor_cells.h5ad'
Cancer_adata = sc.read(input_cancer)

In [3]:
signatures_path = Path("/Users/anna.maguza/Downloads/signatures.gmt")
pd.read_csv(signatures_path, sep="\t", header=None)

,0,1,2,3,4,5,6,7,8,9,...,1305,1306,1307,1308,1309,1310,1311,1312,1313,1314
0,Normal,ikarus,RP11-128M1.1,TRAV8-2,PTPRD-AS1,MEOX2,CXCL12,KLRC4-KLRK1,BCAS1,SCNN1A,...,C22ORF15,CYP4F11,AK8,LRRC18,LMO2,COL12A1,ITGA11,EGFL6,RGS11,PCDHB15
1,Tumor,ikarus,RP11-277P12.10,RP13-895J2.6,BNC1,MAGEA6,ISX,MAGEA3,RP13-614K11.2,CDH7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Make the column in var 'gene

In [5]:
# Filter epithelial cells
Cancer_adata = Cancer_adata[Cancer_adata.obs['Cell Type'] == 'Epithelial',:]

# make the column in var 'gene_symbol' that copies var index
Cancer_adata.var['gene_symbol'] = Cancer_adata.var.index

Cancer_adata = data.preprocess_adata(Cancer_adata)

/var/folders/gb/wwnzrh015q9dns5cmsc3xcjsxglkqz/T/ipykernel_54063/503726983.py:5: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  Cancer_adata.var['gene_symbol'] = Cancer_adata.var.index


In [4]:
model_path = Path("/Users/anna.maguza/Downloads/core_model.joblib")
model = classifier.Ikarus(signatures_gmt=signatures_path, out_dir="/Users/anna.maguza/Desktop/Data/Processed_datasets/Cancer_dataset_integration/output/Ikarus_output")
model.load_core_model(model_path)

/Users/anna.maguza/miniconda3/envs/ikarus_env/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
output = model.predict(adatas["Joanito"], "Joanito", save=True)

/Users/anna.maguza/miniconda3/envs/ikarus_env/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/anna.maguza/miniconda3/envs/ikarus_env/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/anna.maguza/miniconda3/envs/ikarus_env/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:540: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['hvg'] = {'flavor': flavor}
/Users/anna.maguza/miniconda3/envs/ikarus_env/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://n

converged at iteration step: 9 with 0.0007 < 0.001
